# Exploring the Recent Data (dates after January 2018)

The complete dataset kept on crashing my laptop, so I cut down the dataset to dates after January 2018 (including). 

## Setup

In [ ]:
# --- Configture Notebook ------
# show all outputs of cell
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=100,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

# enable automatic reloading
%load_ext autoreload
%autoreload 2

from pathlib import Path

from pandas.core.base import PandasObject
from tends.utils import display_fully

PandasObject.display_fully = display_fully

DATAFOLDER = Path().cwd().parent / "data"

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv(DATAFOLDER / "raw" / "espeni_recent.csv")
df.head()

df.shape

In [ ]:
to_rename = {
    "POWER_ESPENI_MW": "total_demand",
    "POWER_ELEXM_CCGT_MW": "gen_combined_gas",
    "POWER_ELEXM_OIL_MW": "gen_oil",
    "POWER_ELEXM_COAL_MW": "gen_coal",
    "POWER_ELEXM_NUCLEAR_MW": "gen_nuclear",
    "POWER_ELEXM_WIND_MW": "gen_wind",
    "POWER_ELEXM_PS_MW": "gen_pumped",
    "POWER_ELEXM_NPSHYD_MW": "gen_non_pumped",
    "POWER_ELEXM_OCGT_MW": "gen_open_cycle_gas",
    "POWER_ELEXM_OTHER_POSTCALC_MW": "gen_other",
    "POWER_ELEXM_BIOMASS_POSTCALC_MW": "gen_biomass",
    "POWER_NGEM_EMBEDDED_SOLAR_GENERATION_MW": "gen_solar",
    "POWER_NGEM_EMBEDDED_WIND_GENERATION_MW": "gen_embedded_wind",
    "POWER_NGEM_BRIDNET_FLOW_MW": "gen_bridnet",
}

df = df.rename(columns=to_rename)
df.columns = [c.lower() for c in df.columns]
df.head()

In [ ]:
generation_columns = [c for c in df.columns if "gen_" in c]

In [ ]:
df["total_generation"] = df[generation_columns].sum(axis=1)
df["unmet_demand"] = df["total_demand"] - df["total_generation"]

In [ ]:
for cname in sorted(generation_columns):
    px.line(df, x="elexm_utc", y=cname, title=cname)

px.line(df, x="elexm_utc", y="total_generation", title="Total Generation")
px.line(df, x="elexm_utc", y="total_demand", title="Total Demand")
px.line(df, x="elexm_utc", y="unmet_demand", title="Total Demand - Total Generation")

Insights:

* I would have expected the total generation to add up to the same as the total demand. The fact that this is not the case make me think that I  might be misunderstanding the total demand or how to calculate the total generation. In particular, I am not sure how the NGEM data plays into the numbers here. As far as I understand the NGEM data is about energy-transport/exchange with the Netherlands. However, it is not clear to me how the two embedded contributions play into the equation.
* 

In [ ]:
prices = pd.read_csv(DATAFOLDER / "raw" / "electricity-prices-day-a.csv", parse_dates=True)
prices.columns = ["day", "price"]
prices = prices[prices["day"] > "2018-01-01"]

prices.head()

In [ ]:
px.line(prices, x="day", y="price")

## Predicting Demand

In [ ]:
df["month"] = df["elexm_settlement_date"].map(lambda x: x.split("-")[1])

In [ ]:
forecast = (
    df[df["elexm_settlement_date"].map(lambda x: x.split("-")[0]) < "2022"]
    .groupby(["month", "elexm_settlement_period"])[["total_demand"]]
    .mean()
)
forecast

In [ ]:
df = (
    df.set_index(["month", "elexm_settlement_period"])
    .join(forecast, rsuffix="_forecast")
    .reset_index()
)
df.head()
df.shape

In [ ]:
px.line(
    df.sort_values(by=["elexm_settlement_date", "elexm_settlement_period"]),
    x="elexm_utc",
    y=["total_demand", "total_demand_forecast"],
)